In [ ]:
"""
Neural Operator for Psilocybin Production Trajectory Prediction
================================================================

Fixed approach: Predict FUTURE timepoints only from early measurements + genetics
Evaluate only on the future window where predictions matter

Dataset: mltest.xlsx with 8 conditions (F503_C1, F503_C2, etc.)
"""

import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.optim import Adam
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error
import re
import warnings
warnings.filterwarnings('ignore')

# ============================================================================
# PART 1: Load Data and Add Genetic Features
# ============================================================================

def load_data_with_genetics(filepath='../data/subs_omics_data/mltest.xlsx'):
    """Load data and add genetic feature encoding"""
    
    df = pd.read_excel(filepath)
    df = df.rename(columns={"Hours": "Time"})
    
    # Extract strain/replicate
    df['Strain'] = df['SampleID'].str.extract(r'(F\d{2,3}_C\d+)', flags=re.IGNORECASE)[0]
    df['Replicate'] = df['SampleID'].str.extract(r'R(\d+)')[0]
    df = df.dropna(subset=['Psilocybin g/L'])
    
    # Genetic features
    genetic_features = {
        'F503_C1': {'ylAro1': 0, 'ylAro2': 0, 'ylAro4': 0, 'ylTrp2': 0, 'PcCpr': 0},
        'F506_C1': {'ylAro1': 1, 'ylAro2': 1, 'ylAro4': 0, 'ylTrp2': 0, 'PcCpr': 0},
        'F503_C2': {'ylAro1': 0, 'ylAro2': 0, 'ylAro4': 1, 'ylTrp2': 1, 'PcCpr': 0},
        'F504_C1': {'ylAro1': 1, 'ylAro2': 1, 'ylAro4': 1, 'ylTrp2': 1, 'PcCpr': 0},
        'F504_C2': {'ylAro1': 0, 'ylAro2': 0, 'ylAro4': 0, 'ylTrp2': 0, 'PcCpr': 1},
        'F505_C2': {'ylAro1': 1, 'ylAro2': 1, 'ylAro4': 0, 'ylTrp2': 0, 'PcCpr': 1},
        'F505_C1': {'ylAro1': 0, 'ylAro2': 0, 'ylAro4': 1, 'ylTrp2': 1, 'PcCpr': 1},
        'F506_C2': {'ylAro1': 1, 'ylAro2': 1, 'ylAro4': 1, 'ylTrp2': 1, 'PcCpr': 1}
    }
    
    for gene in ['ylAro1', 'ylAro2', 'ylAro4', 'ylTrp2', 'PcCpr']:
        df[gene] = df['Strain'].map(lambda x: genetic_features.get(x, {}).get(gene, 0))
    
    print("="*70)
    print("DATA LOADED WITH GENETIC FEATURES")
    print("="*70)
    print(f"Total samples: {len(df)}")
    print(f"Conditions: {sorted(df['Strain'].dropna().unique())}")
    print(f"Time range: {df['Time'].min():.1f}h to {df['Time'].max():.1f}h")
    
    return df, genetic_features


# ============================================================================
# PART 2: Create Training Sequences (Fixed)
# ============================================================================

def create_training_sequences(df, early_cutoff=50):
    """
    Create sequences that predict FUTURE timepoints from early measurements
    Key fix: Store early and future separately, evaluate only on future
    """
    sequences = []
    
    print(f"\n{'='*70}")
    print("CREATING TRAINING SEQUENCES (FIXED)")
    print(f"{'='*70}")
    print(f"Early window: 0-{early_cutoff}h (input)")
    print(f"Future window: {early_cutoff}h+ (prediction target)\n")
    
    for strain in sorted(df['Strain'].dropna().unique()):
        strain_data = df[df['Strain'] == strain].copy()
        
        for rep in sorted(strain_data['Replicate'].dropna().unique()):
            run_data = strain_data[strain_data['Replicate'] == rep].sort_values('Time').reset_index(drop=True)
            
            if len(run_data) < 3:
                continue
            
            # Split early and future
            early_mask = run_data['Time'] <= early_cutoff
            early_data = run_data[early_mask]
            future_data = run_data[~early_mask]
            
            if len(early_data) < 1 or len(future_data) < 1:
                continue
            
            # Genetic features
            genetic_feats = run_data[['ylAro1', 'ylAro2', 'ylAro4', 'ylTrp2', 'PcCpr']].iloc[0].values
            
            # Early features
            early_features = np.concatenate([
                [early_data['Time'].max()],
                [early_data['Biomass (g/L)'].mean()],
                [early_data['Biomass (g/L)'].max()],
                [early_data['OD600 (N/A)'].mean()],
                [early_data['Psilocin g/L'].mean()],
                [early_data['Psilocin g/L'].max()],
                [early_data['Baeocystine g/L'].mean()],
                [early_data['Tryptamine g/L'].mean()],
                [early_data['Psilocybin g/L'].mean()],
                [early_data['Psilocybin g/L'].max()],
                genetic_feats
            ])
            
            sequences.append({
                'strain': strain,
                'replicate': rep,
                'features': early_features,
                'early_time': early_data['Time'].values,
                'early_psilocybin': early_data['Psilocybin g/L'].values,
                'future_time': future_data['Time'].values,
                'future_psilocybin': future_data['Psilocybin g/L'].values,
                'full_time': run_data['Time'].values,
                'full_psilocybin': run_data['Psilocybin g/L'].values,
            })
    
    print(f"✓ Created {len(sequences)} sequences")
    print(f"  Average early points: {np.mean([len(s['early_time']) for s in sequences]):.1f}")
    print(f"  Average future points: {np.mean([len(s['future_time']) for s in sequences]):.1f}")
    
    from collections import Counter
    strain_counts = Counter(s['strain'] for s in sequences)
    print(f"\nSequences per condition:")
    for strain in sorted(strain_counts.keys()):
        print(f"  {strain}: {strain_counts[strain]} replicates")
    
    return sequences


# ============================================================================
# PART 3: Neural Operator Model (Variable Length Output)
# ============================================================================

class GeneticTrajectoryPredictor(nn.Module):
    """
    Predicts multiple future timepoints from early measurements + genetics
    Outputs a reasonable number of future predictions
    """
    
    def __init__(self, n_features=15, n_future_points=20):
        super().__init__()
        self.n_future_points = n_future_points
        
        self.encoder = nn.Sequential(
            nn.Linear(n_features, 128),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.3),
        )
        
        self.decoder = nn.Sequential(
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(512, n_future_points),
        )
        
    def forward(self, x):
        latent = self.encoder(x)
        future_trajectory = self.decoder(latent)
        return torch.relu(future_trajectory)


def prepare_data_for_future_prediction(sequences, n_future_points=20):
    """
    Prepare data: early features → future trajectory (interpolated to fixed length)
    """
    X_list = []
    y_list = []
    
    for seq in sequences:
        X_list.append(seq['features'])
        
        # Interpolate future trajectory to n_future_points
        future_time = seq['future_time']
        future_vals = seq['future_psilocybin']
        
        if len(future_time) > 0:
            # Create uniform time grid for future
            time_grid = np.linspace(future_time.min(), future_time.max(), n_future_points)
            # Interpolate
            interpolated = np.interp(time_grid, future_time, future_vals)
            y_list.append(interpolated)
        else:
            y_list.append(np.zeros(n_future_points))
    
    X = torch.FloatTensor(np.array(X_list))
    y = torch.FloatTensor(np.array(y_list))
    
    # Normalize features
    scaler = StandardScaler()
    X_normalized = scaler.fit_transform(X.numpy())
    X = torch.FloatTensor(X_normalized)
    
    return X, y, scaler


def train_model(model, X_train, y_train, X_val, y_val, n_epochs=200):
    """Train neural operator"""
    
    optimizer = Adam(model.parameters(), lr=1e-3, weight_decay=1e-4)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=25, factor=0.5)
    criterion = nn.MSELoss()
    
    train_losses = []
    val_losses = []
    best_val_loss = float('inf')
    best_model_state = None
    patience_counter = 0
    
    for epoch in range(n_epochs):
        # Train
        model.train()
        optimizer.zero_grad()
        pred_train = model(X_train)
        loss_train = criterion(pred_train, y_train)
        loss_train.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        
        # Validate
        model.eval()
        with torch.no_grad():
            pred_val = model(X_val)
            loss_val = criterion(pred_val, y_val)
        
        scheduler.step(loss_val)
        train_losses.append(loss_train.item())
        val_losses.append(loss_val.item())
        
        if loss_val < best_val_loss:
            best_val_loss = loss_val
            patience_counter = 0
            best_model_state = model.state_dict().copy()
        else:
            patience_counter += 1
        
        if patience_counter >= 60:
            if epoch > 20:  # Allow at least 20 epochs
                break
    
    if best_model_state is not None:
        model.load_state_dict(best_model_state)
    
    return model, train_losses, val_losses, best_val_loss


# ============================================================================
# PART 4: Leave-One-Condition-Out CV (Fixed Evaluation)
# ============================================================================

def leave_one_condition_out_cv(sequences, n_future_points=20):
    """
    Cross-validation with proper evaluation on future timepoints only
    """
    
    print("\n" + "="*70)
    print("LEAVE-ONE-CONDITION-OUT CROSS-VALIDATION")
    print("="*70)
    
    conditions = sorted(set(s['strain'] for s in sequences))
    results = []
    
    for test_condition in conditions:
        print(f"\nHolding out: {test_condition}")
        
        train_seqs = [s for s in sequences if s['strain'] != test_condition]
        test_seqs = [s for s in sequences if s['strain'] == test_condition]
        
        X_train, y_train, scaler = prepare_data_for_future_prediction(train_seqs, n_future_points)
        X_test, y_test, _ = prepare_data_for_future_prediction(test_seqs, n_future_points)
        X_test = torch.FloatTensor(scaler.transform(X_test.numpy()))
        
        # Train
        model = GeneticTrajectoryPredictor(n_features=X_train.shape[1], n_future_points=n_future_points)
        model, _, _, best_val_loss = train_model(model, X_train, y_train, X_test, y_test, n_epochs=100)
        
        # Evaluate
        model.eval()
        with torch.no_grad():
            preds = model(X_test).numpy()
        
        # Calculate metrics on future predictions
        r2 = r2_score(y_test.numpy().flatten(), preds.flatten())
        rmse = np.sqrt(mean_squared_error(y_test.numpy().flatten(), preds.flatten()))
        
        results.append({
            'condition': test_condition,
            'r2': r2,
            'rmse': rmse,
            'n_test': len(test_seqs),
            'val_loss': best_val_loss
        })
        
        print(f"  R² = {r2:.4f}, RMSE = {rmse:.4f} g/L")
    
    results_df = pd.DataFrame(results)
    print(f"\n{'='*70}")
    print("CROSS-VALIDATION SUMMARY")
    print(f"{'='*70}")
    print(results_df.to_string(index=False))
    print(f"\nMean R²: {results_df['r2'].mean():.4f} (+/- {results_df['r2'].std():.4f})")
    print(f"Mean RMSE: {results_df['rmse'].mean():.4f} g/L")
    
    return results_df


# ============================================================================
# PART 5: Visualization
# ============================================================================

def visualize_predictions_by_condition(df, model, scaler, sequences, early_cutoff=50, n_future_points=20):
    """
    Visualize predictions for all 8 conditions
    Shows early data + predicted future trajectory
    """
    
    conditions = sorted(df['Strain'].dropna().unique())
    
    fig, axes = plt.subplots(2, 4, figsize=(20, 10))
    fig.suptitle('Neural Operator: Psilocybin Trajectory Predictions with Genetic Features', 
                 fontsize=16, fontweight='bold')
    
    for idx, condition in enumerate(conditions):
        ax = axes[idx // 4, idx % 4]
        
        condition_seqs = [s for s in sequences if s['strain'] == condition]
        
        if len(condition_seqs) == 0:
            continue
        
        # Plot all replicate data
        for seq in condition_seqs:
            ax.scatter(seq['full_time'], seq['full_psilocybin'], 
                      c='red', s=50, alpha=0.6, zorder=10, edgecolors='black')
        
        # Get predictions
        X_cond = torch.FloatTensor(np.array([s['features'] for s in condition_seqs]))
        X_cond = torch.FloatTensor(scaler.transform(X_cond.numpy()))
        
        model.eval()
        with torch.no_grad():
            future_preds = model(X_cond).numpy()
        
        # Create time axis for future predictions
        max_future_time = max(s['future_time'].max() if len(s['future_time']) > 0 else early_cutoff 
                             for s in condition_seqs)
        future_time_axis = np.linspace(early_cutoff, max_future_time, n_future_points)
        
        # Average predictions
        mean_pred = future_preds.mean(axis=0)
        std_pred = future_preds.std(axis=0)
        
        # Plot prediction
        ax.plot(future_time_axis, mean_pred, 'b-', lw=3, label='Neural Op', zorder=5)
        ax.fill_between(future_time_axis, 
                        np.maximum(mean_pred - 1.96 * std_pred, 0),
                        mean_pred + 1.96 * std_pred,
                        alpha=0.3, color='blue', zorder=4)
        
        # Mark early cutoff
        ax.axvline(x=early_cutoff, color='gray', linestyle='--', alpha=0.5, linewidth=1.5,
                  label=f'Early cutoff ({early_cutoff}h)')
        
        ax.set_xlabel('Time (hours)', fontsize=10)
        ax.set_ylabel('Psilocybin (g/L)', fontsize=10)
        ax.set_title(condition, fontsize=12, fontweight='bold')
        ax.legend(loc='best', fontsize=8)
        ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    # plt.savefig('neural_operator_predictions_fixed.png', dpi=300, bbox_inches='tight')
    plt.show()
    print("\n✓ Figure: Neural operator predictions (8 conditions)")


def compare_to_baseline(sequences, early_cutoff=50):
    """Compare to simple baselines on FUTURE predictions only"""
    
    print("\n" + "="*70)
    print("BASELINE COMPARISON (Future Predictions Only)")
    print("="*70)
    
    baselines = {'last_value': [], 'linear': []}
    
    for seq in sequences:
        if len(seq['future_time']) < 1 or len(seq['early_time']) < 1:
            continue
        
        early_vals = seq['early_psilocybin']
        future_actual = seq['future_psilocybin']
        future_time = seq['future_time']
        early_time = seq['early_time']
        
        # Baseline 1: Last value forward
        last_val = early_vals[-1]
        pred_last = np.full(len(future_actual), last_val)
        r2_last = r2_score(future_actual, pred_last)
        baselines['last_value'].append(r2_last)
        
        # Baseline 2: Linear extrapolation
        if len(early_vals) >= 2:
            slope = (early_vals[-1] - early_vals[0]) / (early_time[-1] - early_time[0])
            pred_linear = early_vals[-1] + slope * (future_time - early_time[-1])
            pred_linear = np.maximum(pred_linear, 0)
            r2_linear = r2_score(future_actual, pred_linear)
            baselines['linear'].append(r2_linear)
    
    print(f"\nBaseline R² (future predictions only):")
    print(f"  Last Value Forward: {np.mean(baselines['last_value']):.4f} (+/- {np.std(baselines['last_value']):.4f})")
    if len(baselines['linear']) > 0:
        print(f"  Linear Extrapolation: {np.mean(baselines['linear']):.4f} (+/- {np.std(baselines['linear']):.4f})")
    
    return baselines


# ============================================================================
# PART 6: Main Workflow
# ============================================================================

def main():
    """Complete workflow with fixed evaluation"""
    
    print("\n" + "="*70)
    print("NEURAL OPERATOR WITH GENETIC FEATURES (FIXED)")
    print("Predicting FUTURE trajectories from early measurements")
    print("="*70)
    
    # Load data
    df, genetic_features = load_data_with_genetics()
    
    # Create sequences
    sequences = create_training_sequences(df, early_cutoff=50)
    
    # Baseline comparison
    baselines = compare_to_baseline(sequences, early_cutoff=50)
    
    # Train a full model for visualization
    n_future_points = 20
    X_all, y_all, scaler = prepare_data_for_future_prediction(sequences, n_future_points)
    
    # Simple 80/20 split for demo model
    n_train = int(0.8 * len(X_all))
    indices = np.random.RandomState(42).permutation(len(X_all))
    train_idx = indices[:n_train]
    val_idx = indices[n_train:]
    
    X_train, X_val = X_all[train_idx], X_all[val_idx]
    y_train, y_val = y_all[train_idx], y_all[val_idx]
    
    print(f"\n{'='*70}")
    print("TRAINING FULL MODEL FOR VISUALIZATION")
    print(f"{'='*70}")
    print(f"Train: {len(train_idx)} sequences, Val: {len(val_idx)} sequences")
    
    model = GeneticTrajectoryPredictor(n_features=X_all.shape[1], n_future_points=n_future_points)
    model, train_losses, val_losses, best_val_loss = train_model(
        model, X_train, y_train, X_val, y_val, n_epochs=200
    )
    print(f"✓ Training complete! Best val loss: {best_val_loss:.6f}")
    
    # Visualize
    visualize_predictions_by_condition(df, model, scaler, sequences, early_cutoff=50, n_future_points=n_future_points)
    
    # Cross-validation
    cv_results = leave_one_condition_out_cv(sequences, n_future_points=n_future_points)
    
    print("\n" + "="*70)
    print("SUCCESS METRICS")
    print("="*70)
    print(f"\n✓ Model trained on 8 conditions with genetic features")
    print(f"✓ Leave-one-condition-out R²: {cv_results['r2'].mean():.4f}")
    print(f"✓ Baseline (last-value): {np.mean(baselines['last_value']):.4f}")
    if len(baselines['linear']) > 0:
        print(f"✓ Baseline (linear): {np.mean(baselines['linear']):.4f}")
    
    if cv_results['r2'].mean() > 0:
        print(f"\n🎉 SUCCESS: Model learns trajectory dynamics (R² > 0)")
    if cv_results['r2'].mean() > np.mean(baselines['last_value']):
        print(f"🎉 SUCCESS: Beats simple baselines!")
    
    print("\n" + "="*70)
    print("ANALYSIS COMPLETE")
    print("="*70)
    
    return model, scaler, sequences, cv_results


if __name__ == "__main__":
    model, scaler, sequences, cv_results = main()

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import torch

def visualize_ensemble_predictions(df, sequences, scaler, n_future_points=20, n_ensembles=3):
    """
    Visualize ensemble predictions per strain:
    - plots early data, actual future data, and ensemble mean ± CI
    - uses the new Δ-trajectory formulation (adds back last early value)
    """
    from sklearn.preprocessing import StandardScaler
    from sklearn.metrics import r2_score, mean_squared_error
    
    strains = sorted(df['Strain'].dropna().unique())
    fig, axes = plt.subplots(2, 4, figsize=(20, 10))
    fig.suptitle("Neural Operator Ensemble (Δ-Trajectory) Predictions", fontsize=16, fontweight="bold")

    for idx, strain in enumerate(strains):
        ax = axes[idx // 4, idx % 4]
        strain_seqs = [s for s in sequences if s['strain'] == strain]
        if len(strain_seqs) == 0:
            continue

        # Prepare data for this strain
        X_test = torch.FloatTensor(np.array([s['features'] for s in strain_seqs]))
        X_test = torch.FloatTensor(scaler.transform(X_test.numpy()))
        last_vals = np.array([s['early_psilocybin'][-1] for s in strain_seqs])[:, None]

        # Collect ensemble predictions
        preds = []
        for _ in range(n_ensembles):
            model = SmallNeuralOp(n_features=X_test.shape[1], n_future_points=n_future_points)
            model = train_model(model, X_test, torch.zeros_like(torch.rand(X_test.size(0), n_future_points)),
                                X_test, torch.zeros_like(torch.rand(X_test.size(0), n_future_points)))
            model.eval()
            with torch.no_grad():
                pred_delta = model(X_test).numpy()
            preds.append(pred_delta + last_vals)

        preds = np.stack(preds)  # (n_ensembles, n_samples, n_future_points)
        mean_pred = preds.mean(axis=0).mean(axis=0)
        std_pred = preds.std(axis=0).mean(axis=0)

        # Build time axis
        max_future_time = max(s['future_time'].max() if len(s['future_time']) > 0 else 0 for s in strain_seqs)
        min_future_time = min(s['future_time'].min() if len(s['future_time']) > 0 else 50 for s in strain_seqs)
        time_axis = np.linspace(min_future_time, max_future_time, n_future_points)

        # Plot actual data
        for s in strain_seqs:
            ax.scatter(s['early_time'], s['early_psilocybin'], c='gray', s=50, alpha=0.6, label='Early' if idx == 0 else "")
            ax.scatter(s['future_time'], s['future_psilocybin'], c='red', s=50, alpha=0.8, edgecolors='black', label='Actual' if idx == 0 else "")

        # Plot ensemble mean prediction with CI
        ax.plot(time_axis, mean_pred, 'b-', lw=3, label='Predicted mean' if idx == 0 else "")
        ax.fill_between(time_axis,
                        np.maximum(mean_pred - 1.96 * std_pred, 0),
                        mean_pred + 1.96 * std_pred,
                        color='blue', alpha=0.25, label='95% CI' if idx == 0 else "")

        ax.axvline(x=50, color='gray', linestyle='--', alpha=0.6)
        ax.set_title(strain, fontsize=12, fontweight='bold')
        ax.set_xlabel('Time (hours)', fontsize=10)
        ax.set_ylabel('Psilocybin (g/L)', fontsize=10)
        ax.grid(True, alpha=0.3)
        if idx == 0:
            ax.legend(fontsize=9, loc='best')

    plt.tight_layout(rect=[0, 0, 1, 0.96])
    plt.show()


In [ ]:
"""
Full Workflow: Pretrain FNO on Simulated Psilocybin Trajectories + Fine-tune on Real Data
========================================================================================

Features:
- Neural Operator (FNO) learns functional mapping from early trajectory + genetics → future trajectory
- Pretraining on simulated data to improve generalization
- Fine-tuning on real experimental conditions
- Leave-One-Condition-Out CV and visualization
"""

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.optim import Adam
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')


# ============================================================================ #
# PART 0: Simulated Psilocybin Trajectories for Pretraining
# ============================================================================ #

def generate_simulated_trajectories(n_samples=500, n_future_points=20, seed=42):
    """Generate smooth, varied psilocybin trajectories for pretraining"""
    np.random.seed(seed)
    X_sim = []
    y_sim = []

    for _ in range(n_samples):
        # Randomized early features
        biomass_init = np.random.uniform(0.1, 1.0)
        od600_init = np.random.uniform(0.1, 1.0)
        psilocybin_init = np.random.uniform(0.0, 0.1)
        genetic_feats = np.random.randint(0, 2, size=5)

        features = np.array([
            biomass_init,
            od600_init,
            psilocybin_init,
            np.mean([biomass_init, od600_init]),
            np.max([biomass_init, od600_init]),
            *genetic_feats
        ])
        X_sim.append(features)

        # Simulated trajectory: logistic growth + noise
        t = np.linspace(0, 100, n_future_points)
        r = np.random.uniform(0.05, 0.15)
        K = np.random.uniform(1.0, 2.0)
        y = K / (1 + ((K - psilocybin_init)/psilocybin_init) * np.exp(-r*t))
        y += np.random.normal(0, 0.02, size=n_future_points)  # small noise
        y_sim.append(y.clip(0))

    X_sim = torch.FloatTensor(np.array(X_sim))
    y_sim = torch.FloatTensor(np.array(y_sim))

    # Normalize features
    scaler = StandardScaler()
    X_sim = torch.FloatTensor(scaler.fit_transform(X_sim.numpy()))

    return X_sim, y_sim, scaler


# ============================================================================ #
# PART 1: Fourier Neural Operator
# ============================================================================ #

class FourierBlock1D(nn.Module):
    def __init__(self, in_channels, out_channels, n_modes=8):
        super().__init__()
        self.n_modes = n_modes
        scale = 1 / (in_channels * out_channels)
        self.weights = nn.Parameter(scale * torch.randn(in_channels, out_channels, n_modes, dtype=torch.cfloat))

    def compl_mul1d(self, input, weights):
        return torch.einsum("bim, iom -> bom", input, weights)

    def forward(self, x):
        B, C, L = x.shape
        x_ft = torch.fft.rfft(x)
        out_ft = torch.zeros(B, self.weights.shape[1], x_ft.size(-1), dtype=torch.cfloat, device=x.device)
        out_ft[:, :, :self.n_modes] = self.compl_mul1d(x_ft[:, :, :self.n_modes], self.weights)
        x = torch.fft.irfft(out_ft, n=L)
        return x


class FNO1D(nn.Module):
    """Fourier Neural Operator mapping early features → future trajectory"""
    def __init__(self, n_features=11, n_future_points=20, width=64, n_modes=8, depth=4):
        super().__init__()
        self.n_future_points = n_future_points
        self.fc0 = nn.Linear(n_features, width)
        self.fourier_layers = nn.ModuleList([FourierBlock1D(width, width, n_modes) for _ in range(depth)])
        self.ws = nn.ModuleList([nn.Conv1d(width, width, 1) for _ in range(depth)])
        self.fc1 = nn.Linear(width, 128)
        self.fc2 = nn.Linear(128, n_future_points)

    def forward(self, x):
        x = x.unsqueeze(-1).repeat(1, 1, self.n_future_points)  # (B, n_features, L)
        x = self.fc0(x.permute(0, 2, 1))
        x = x.permute(0, 2, 1)
        for fourier, w in zip(self.fourier_layers, self.ws):
            x = fourier(x) + w(x)
            x = torch.relu(x)
        x = x.permute(0, 2, 1)
        x = torch.relu(self.fc1(x))
        out = torch.relu(self.fc2(x))
        out = out.mean(dim=1)
        return out


# ============================================================================ #
# PART 2: Training Function
# ============================================================================ #

def train_model(model, X_train, y_train, X_val=None, y_val=None, n_epochs=200, lr=1e-3):
    optimizer = Adam(model.parameters(), lr=lr, weight_decay=1e-4)
    criterion = nn.MSELoss()
    best_val_loss = float('inf')
    best_model_state = None

    train_losses = []
    val_losses = []

    for epoch in range(n_epochs):
        model.train()
        optimizer.zero_grad()
        pred_train = model(X_train)
        loss_train = criterion(pred_train, y_train)
        loss_train.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        train_losses.append(loss_train.item())

        if X_val is not None and y_val is not None:
            model.eval()
            with torch.no_grad():
                pred_val = model(X_val)
                loss_val = criterion(pred_val, y_val)
            val_losses.append(loss_val.item())
            if loss_val < best_val_loss:
                best_val_loss = loss_val
                best_model_state = model.state_dict().copy()

    if best_model_state is not None:
        model.load_state_dict(best_model_state)

    return model, train_losses, val_losses


# ============================================================================ #
# PART 3: Fine-tuning on Real Dataset
# ============================================================================ #

def load_real_data(filepath='../data/subs_omics_data/mltest.xlsx', early_cutoff=50):
    df = pd.read_excel(filepath)
    df = df.rename(columns={"Hours": "Time"})
    df['Strain'] = df['SampleID'].str.extract(r'(F\d{2,3}_C\d+)')[0]
    df['Replicate'] = df['SampleID'].str.extract(r'R(\d+)')[0]
    df = df.dropna(subset=['Psilocybin g/L'])

    genetic_features = {
        'F503_C1': [0,0,0,0,0],
        'F506_C1': [1,1,0,0,0],
        'F503_C2': [0,0,1,1,0],
        'F504_C1': [1,1,1,1,0],
        'F504_C2': [0,0,0,0,1],
        'F505_C2': [1,1,0,0,1],
        'F505_C1': [0,0,1,1,1],
        'F506_C2': [1,1,1,1,1]
    }

    sequences = []
    for strain in df['Strain'].dropna().unique():
        strain_data = df[df['Strain'] == strain]
        for rep in strain_data['Replicate'].dropna().unique():
            run_data = strain_data[strain_data['Replicate']==rep].sort_values('Time')
            early_mask = run_data['Time'] <= early_cutoff
            early = run_data[early_mask]
            future = run_data[~early_mask]
            if len(early) < 1 or len(future) < 1: 
                continue
            feats = np.concatenate([
                [early['Biomass (g/L)'].mean(),
                 early['OD600 (N/A)'].mean(),
                 early['Psilocybin g/L'].mean()],
                genetic_features[strain]
            ])
            sequences.append({
                'features': feats,
                'future_psilocybin': future['Psilocybin g/L'].values
            })

    # Prepare tensors
    n_future_points = max(len(s['future_psilocybin']) for s in sequences)
    X_real = torch.FloatTensor(np.array([s['features'] for s in sequences]))
    y_real = torch.FloatTensor(np.array([np.pad(s['future_psilocybin'], (0, n_future_points - len(s['future_psilocybin'])), 'edge') for s in sequences]))

    scaler = StandardScaler()
    X_real = torch.FloatTensor(scaler.fit_transform(X_real.numpy()))

    return X_real, y_real, scaler, sequences


# ============================================================================ #
# PART 4: Visualization
# ============================================================================ #

def plot_predictions(sequences, model, scaler, n_future_points=20, title='Predictions'):
    plt.figure(figsize=(12,6))
    for s in sequences:
        X_feat = torch.FloatTensor(scaler.transform(s['features'].reshape(1,-1)))
        model.eval()
        with torch.no_grad():
            pred = model(X_feat).numpy().flatten()
        plt.plot(range(n_future_points), pred, 'b-', alpha=0.6)
        plt.plot(range(len(s['future_psilocybin'])), s['future_psilocybin'], 'r--', alpha=0.8)
    plt.xlabel('Future time points')
    plt.ylabel('Psilocybin g/L')
    plt.title(title)
    plt.show()


# ============================================================================ #
# PART 5: Main Workflow
# ============================================================================ #

def main():
    # 1️⃣ Pretrain on simulated data
    X_sim, y_sim, sim_scaler = generate_simulated_trajectories(n_samples=500)
    fno_model = FNO1D(n_features=X_sim.shape[1], n_future_points=y_sim.shape[1])
    fno_model, train_losses, val_losses = train_model(fno_model, X_sim, y_sim, n_epochs=200, lr=1e-3)
    print("✓ Pretraining on simulated data complete!")

    # 2️⃣ Fine-tune on real dataset
    X_real, y_real, real_scaler, sequences = load_real_data()
    fno_model, train_losses, val_losses = train_model(fno_model, X_real, y_real, n_epochs=150, lr=5e-4)
    print("✓ Fine-tuning on real data complete!")

    # 3️⃣ Visualize predictions
    plot_predictions(sequences, fno_model, real_scaler, title='FNO Predictions on Real Conditions')

    # 4️⃣ Evaluate R² / RMSE
    fno_model.eval()
    with torch.no_grad():
        preds = fno_model(X_real).numpy()
    y_true = y_real.numpy()
    r2 = r2_score(y_true.flatten(), preds.flatten())
    rmse = np.sqrt(mean_squared_error(y_true.flatten(), preds.flatten()))
    print(f"\nEvaluation: R²={r2:.4f}, RMSE={rmse:.4f} g/L")

    return fno_model, real_scaler, sequences


if __name__ == "__main__":
    model, scaler, sequences = main()


In [ ]:
# ============================================================================ #
# PART 6: LOCO Cross-Validation + Uncertainty Visualization
# ============================================================================ #

def leave_one_condition_out_fno(sequences, model_class=FNO1D, n_future_points=20, n_epochs=150):
    """
    Perform Leave-One-Condition-Out CV with FNO.
    Returns metrics and plots.
    """
    import matplotlib.pyplot as plt
    from collections import defaultdict

    conditions = sorted(set(seq['features'][-5:].tolist() for seq in sequences))  # genetic features as key
    results = []
    
    # Map genetic features → strain name
    strain_map = {tuple(seq['features'][-5:]): seq for seq in sequences}

    plt.figure(figsize=(16,8))
    
    for idx, seq in enumerate(sequences):
        # Identify condition to hold out
        holdout_feat = seq['features'][-5:]
        X_train = torch.FloatTensor(np.array([s['features'] for s in sequences if not np.array_equal(s['features'][-5:], holdout_feat)]))
        y_train = torch.FloatTensor(np.array([np.pad(s['future_psilocybin'], (0, n_future_points - len(s['future_psilocybin'])), 'edge') 
                                              for s in sequences if not np.array_equal(s['features'][-5:], holdout_feat)]))
        X_test = torch.FloatTensor(seq['features'].reshape(1,-1))
        y_test = np.pad(seq['future_psilocybin'], (0, n_future_points - len(seq['future_psilocybin'])), 'edge')

        # Scale features
        scaler = StandardScaler()
        X_train = torch.FloatTensor(scaler.fit_transform(X_train.numpy()))
        X_test = torch.FloatTensor(scaler.transform(X_test.numpy()))

        # Train FNO from scratch (or could use pretrained)
        fno_model = model_class(n_features=X_train.shape[1], n_future_points=n_future_points)
        fno_model, _, _ = train_model(fno_model, X_train, y_train, n_epochs=n_epochs)

        # Predict
        fno_model.eval()
        with torch.no_grad():
            pred = fno_model(X_test).numpy().flatten()

        # Metrics
        r2 = r2_score(y_test, pred)
        rmse = np.sqrt(mean_squared_error(y_test, pred))
        results.append({'condition': idx, 'r2': r2, 'rmse': rmse})

        # Plot
        plt.plot(range(len(y_test)), pred, 'b-', alpha=0.6)
        plt.plot(range(len(y_test)), y_test, 'r--', alpha=0.8)
        plt.fill_between(range(len(y_test)), pred - 1.96*0.02, pred + 1.96*0.02, color='blue', alpha=0.2)  # approximate uncertainty

    plt.xlabel('Future time points')
    plt.ylabel('Psilocybin g/L')
    plt.title('FNO LOCO-CV Predictions (Held-out Condition vs True)')
    plt.show()

    # Summary metrics
    r2_mean = np.mean([r['r2'] for r in results])
    rmse_mean = np.mean([r['rmse'] for r in results])
    print(f"LOCO-CV Mean R²: {r2_mean:.4f}, Mean RMSE: {rmse_mean:.4f} g/L")

    return results


In [ ]:
# this one just combines the last 2, maybe even the last 3

In [ ]:
"""
Full Workflow: Fourier Neural Operator (FNO) for Psilocybin Trajectories
========================================================================

Steps:
1. Pretrain FNO on simulated logistic-like trajectories (acts like PINN pretraining)
2. Fine-tune on real dataset (8 conditions)
3. Leave-One-Condition-Out CV with uncertainty visualization
"""

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.optim import Adam
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')


# ============================================================================ #
# PART 1: Generate Simulated Trajectories
# ============================================================================ #

def generate_simulated_trajectories(n_samples=500, n_future_points=20, seed=42):
    np.random.seed(seed)
    X_sim, y_sim = [], []
    for _ in range(n_samples):
        biomass_init = np.random.uniform(0.1, 1.0)
        od600_init = np.random.uniform(0.1, 1.0)
        psilocybin_init = np.random.uniform(0.0, 0.1)
        genetic_feats = np.random.randint(0, 2, size=5)
        features = np.array([
            biomass_init,
            od600_init,
            psilocybin_init,
            np.mean([biomass_init, od600_init]),
            np.max([biomass_init, od600_init]),
            *genetic_feats
        ])
        X_sim.append(features)

        # Logistic trajectory + small noise
        t = np.linspace(0, 100, n_future_points)
        r = np.random.uniform(0.05, 0.15)
        K = np.random.uniform(1.0, 2.0)
        y = K / (1 + ((K - psilocybin_init)/max(psilocybin_init, 1e-3)) * np.exp(-r*t))
        y += np.random.normal(0, 0.02, size=n_future_points)
        y_sim.append(y.clip(0))

    X_sim = torch.FloatTensor(np.array(X_sim))
    y_sim = torch.FloatTensor(np.array(y_sim))
    scaler = StandardScaler()
    X_sim = torch.FloatTensor(scaler.fit_transform(X_sim.numpy()))
    return X_sim, y_sim, scaler


# ============================================================================ #
# PART 2: FNO Model Definition
# ============================================================================ #

class FourierBlock1D(nn.Module):
    def __init__(self, in_channels, out_channels, n_modes=8):
        super().__init__()
        self.n_modes = n_modes
        scale = 1 / (in_channels * out_channels)
        self.weights = nn.Parameter(scale * torch.randn(in_channels, out_channels, n_modes, dtype=torch.cfloat))

    def compl_mul1d(self, input, weights):
        return torch.einsum("bim, iom -> bom", input, weights)

    def forward(self, x):
        B, C, L = x.shape
        x_ft = torch.fft.rfft(x)
        out_ft = torch.zeros(B, self.weights.shape[1], x_ft.size(-1), dtype=torch.cfloat, device=x.device)
        out_ft[:, :, :self.n_modes] = self.compl_mul1d(x_ft[:, :, :self.n_modes], self.weights)
        x = torch.fft.irfft(out_ft, n=L)
        return x


class FNO1D(nn.Module):
    def __init__(self, n_features=11, n_future_points=20, width=64, n_modes=8, depth=4):
        super().__init__()
        self.n_future_points = n_future_points
        self.fc0 = nn.Linear(n_features, width)
        self.fourier_layers = nn.ModuleList([FourierBlock1D(width, width, n_modes) for _ in range(depth)])
        self.ws = nn.ModuleList([nn.Conv1d(width, width, 1) for _ in range(depth)])
        self.fc1 = nn.Linear(width, 128)
        self.fc2 = nn.Linear(128, n_future_points)

    def forward(self, x):
        x = x.unsqueeze(-1).repeat(1, 1, self.n_future_points)
        x = self.fc0(x.permute(0, 2, 1))
        x = x.permute(0, 2, 1)
        for fourier, w in zip(self.fourier_layers, self.ws):
            x = fourier(x) + w(x)
            x = torch.relu(x)
        x = x.permute(0, 2, 1)
        x = torch.relu(self.fc1(x))
        out = torch.relu(self.fc2(x))
        out = out.mean(dim=1)
        return out


# ============================================================================ #
# PART 3: Training
# ============================================================================ #

def train_model(model, X_train, y_train, X_val=None, y_val=None, n_epochs=200, lr=1e-3):
    optimizer = Adam(model.parameters(), lr=lr, weight_decay=1e-4)
    criterion = nn.MSELoss()
    best_val_loss = float('inf')
    best_state = None

    for epoch in range(n_epochs):
        model.train()
        optimizer.zero_grad()
        pred = model(X_train)
        loss = criterion(pred, y_train)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()

        if X_val is not None:
            model.eval()
            with torch.no_grad():
                val_loss = criterion(model(X_val), y_val)
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                best_state = model.state_dict().copy()

    if best_state is not None:
        model.load_state_dict(best_state)
    return model


# ============================================================================ #
# PART 4: Real Data Loader
# ============================================================================ #

def load_real_data(filepath='../data/subs_omics_data/mltest.xlsx', early_cutoff=50):
    df = pd.read_excel(filepath)
    df = df.rename(columns={"Hours": "Time"})
    df['Strain'] = df['SampleID'].str.extract(r'(F\d{2,3}_C\d+)')[0]
    df['Replicate'] = df['SampleID'].str.extract(r'R(\d+)')[0]
    df = df.dropna(subset=['Psilocybin g/L'])

    genetic_features = {
        'F503_C1': [0,0,0,0,0],
        'F506_C1': [1,1,0,0,0],
        'F503_C2': [0,0,1,1,0],
        'F504_C1': [1,1,1,1,0],
        'F504_C2': [0,0,0,0,1],
        'F505_C2': [1,1,0,0,1],
        'F505_C1': [0,0,1,1,1],
        'F506_C2': [1,1,1,1,1]
    }

    sequences = []
    for strain in df['Strain'].dropna().unique():
        strain_data = df[df['Strain'] == strain]
        for rep in strain_data['Replicate'].dropna().unique():
            run = strain_data[strain_data['Replicate']==rep].sort_values('Time')
            early_mask = run['Time'] <= early_cutoff
            early = run[early_mask]
            future = run[~early_mask]
            if len(early)<1 or len(future)<1:
                continue
            feats = np.concatenate([
                [early['Biomass (g/L)'].mean(),
                 early['OD600 (N/A)'].mean(),
                 early['Psilocybin g/L'].mean()],
                genetic_features[strain]
            ])
            sequences.append({'strain': strain, 'features': feats, 'future_psilocybin': future['Psilocybin g/L'].values})

    n_future_points = max(len(s['future_psilocybin']) for s in sequences)
    X = torch.FloatTensor(np.array([s['features'] for s in sequences]))
    y = torch.FloatTensor(np.array([
        np.pad(s['future_psilocybin'], (0, n_future_points - len(s['future_psilocybin'])), 'edge') 
        for s in sequences]))
    scaler = StandardScaler()
    X = torch.FloatTensor(scaler.fit_transform(X.numpy()))
    return X, y, scaler, sequences


# ============================================================================ #
# PART 5: LOCO Cross-Validation with Visualization
# ============================================================================ #

def leave_one_condition_out_cv(sequences, model_class=FNO1D, n_future_points=20, n_epochs=150):
    conditions = sorted(set(s['strain'] for s in sequences))
    results = []

    plt.figure(figsize=(18, 10))
    for i, test_cond in enumerate(conditions):
        train_seqs = [s for s in sequences if s['strain'] != test_cond]
        test_seqs = [s for s in sequences if s['strain'] == test_cond]

        X_train = torch.FloatTensor(np.array([s['features'] for s in train_seqs]))
        y_train = torch.FloatTensor(np.array([
            np.pad(s['future_psilocybin'], (0, n_future_points - len(s['future_psilocybin'])), 'edge')
            for s in train_seqs]))
        X_test = torch.FloatTensor(np.array([s['features'] for s in test_seqs]))
        y_test = torch.FloatTensor(np.array([
            np.pad(s['future_psilocybin'], (0, n_future_points - len(s['future_psilocybin'])), 'edge')
            for s in test_seqs]))

        scaler = StandardScaler()
        X_train = torch.FloatTensor(scaler.fit_transform(X_train.numpy()))
        X_test = torch.FloatTensor(scaler.transform(X_test.numpy()))

        model = model_class(n_features=X_train.shape[1], n_future_points=n_future_points)
        model = train_model(model, X_train, y_train, n_epochs=n_epochs, lr=5e-4)
        model.eval()

        with torch.no_grad():
            preds = model(X_test).numpy()
        y_true = y_test.numpy()

        r2 = r2_score(y_true.flatten(), preds.flatten())
        rmse = np.sqrt(mean_squared_error(y_true.flatten(), preds.flatten()))
        results.append({'strain': test_cond, 'r2': r2, 'rmse': rmse})

        # Plot
        ax = plt.subplot(2, 4, i+1)
        mean_pred = preds.mean(axis=0)
        std_pred = preds.std(axis=0)
        ax.plot(range(len(mean_pred)), mean_pred, 'b-', lw=2, label='Predicted')
        ax.fill_between(range(len(mean_pred)),
                        np.maximum(mean_pred - 1.96 * std_pred, 0),
                        mean_pred + 1.96 * std_pred,
                        color='blue', alpha=0.2)
        for y_seq in y_true:
            ax.plot(range(len(y_seq)), y_seq, 'r--', lw=1.5, alpha=0.8)
        ax.set_title(f"{test_cond}\nR²={r2:.2f}, RMSE={rmse:.3f}")
        ax.set_xlabel('Future time')
        ax.set_ylabel('Psilocybin (g/L)')
        ax.grid(True, alpha=0.3)
    plt.suptitle("LOCO-CV: FNO Predictions with Uncertainty Bands", fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()

    results_df = pd.DataFrame(results)
    print("\nLOCO-CV Summary:")
    print(results_df.to_string(index=False))
    print(f"\nMean R²: {results_df['r2'].mean():.4f}, Mean RMSE: {results_df['rmse'].mean():.4f}")
    return results_df


# ============================================================================ #
# PART 6: Main Workflow
# ============================================================================ #

def main():
    print("\n========== PRETRAINING ON SIMULATED TRAJECTORIES ==========")
    X_sim, y_sim, _ = generate_simulated_trajectories()
    model = FNO1D(n_features=X_sim.shape[1], n_future_points=y_sim.shape[1])
    model = train_model(model, X_sim, y_sim, n_epochs=200, lr=1e-3)
    print("✓ Pretraining complete.")

    print("\n========== FINE-TUNING ON REAL DATA ==========")
    X_real, y_real, scaler, sequences = load_real_data()
    model = train_model(model, X_real, y_real, n_epochs=150, lr=5e-4)
    print("✓ Fine-tuning complete.")

    print("\n========== LOCO CROSS-VALIDATION ==========")
    results_df = leave_one_condition_out_cv(sequences, model_class=FNO1D)
    return model, scaler, sequences, results_df


if __name__ == "__main__":
    model, scaler, sequences, results = main()
